<a href="https://colab.research.google.com/github/shajivmasters/Thesis/blob/main/1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
import time
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder
import torch

# Load the dataset
data = pd.read_csv("customer.csv")
# Take a sample of 500 rows
data = data.sample(n=200, random_state=42)
# Preprocess the data
data = data.dropna(subset=['Ticket Description'])  # Drop rows with missing values in the target column
data = data[['Ticket Description', 'Ticket Type']]  # Select relevant columns

# Split the data into train and test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(data['Ticket Description'], data['Ticket Type'], test_size=0.2, random_state=42)

# Encode labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Tokenization
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True)

# Convert encodings to a format suitable for the trainer
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create datasets
train_dataset = MyDataset(train_encodings, train_labels_encoded)
eval_dataset = MyDataset(test_encodings, test_labels_encoded)

# Define BERT model (Pre-trained)
model_pretrained = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_encoder.classes_))

# Define training arguments for pre-trained model
training_args_pretrained = TrainingArguments(
    output_dir='./results_pretrained',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy='epoch',
    logging_dir='./logs_pretrained',
    num_train_epochs=3,
    learning_rate=2e-5,
    save_strategy='epoch',  # Set save strategy to 'epoch'
    load_best_model_at_end=True
)

# Define trainer for pre-trained model
trainer_pretrained = Trainer(
    model=model_pretrained,
    args=training_args_pretrained,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)

# Train and evaluate pre-trained model
start_time = time.time()
trainer_pretrained.train()
preds_pretrained = trainer_pretrained.predict(eval_dataset)
predictions_pretrained = label_encoder.inverse_transform(preds_pretrained.predictions.argmax(axis=1))
accuracy_pretrained = accuracy_score(test_labels, predictions_pretrained)
precision_pretrained = precision_score(test_labels, predictions_pretrained, average='weighted')
recall_pretrained = recall_score(test_labels, predictions_pretrained, average='weighted')
f1_pretrained = f1_score(test_labels, predictions_pretrained, average='weighted')
pretrained_processing_time = time.time() - start_time

# Define BERT model (Fine-tuned by skipping intermediate layers)
model_finetuned = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_encoder.classes_))

# Fine-tune by skipping intermediate layers
model_finetuned.bert.encoder.layer = model_finetuned.bert.encoder.layer[:-2]

# Define training arguments for fine-tuned model
training_args_finetuned = TrainingArguments(
    output_dir='./results_finetuned',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy='epoch',
    logging_dir='./logs_finetuned',
    num_train_epochs=3,
    learning_rate=2e-5,
    save_strategy='epoch',  # Set save strategy to 'epoch'
    load_best_model_at_end=True
)

# Define trainer for fine-tuned model
trainer_finetuned = Trainer(
    model=model_finetuned,
    args=training_args_finetuned,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)

# Train and evaluate fine-tuned model
start_time = time.time()
trainer_finetuned.train()
preds_finetuned = trainer_finetuned.predict(eval_dataset)
predictions_finetuned = label_encoder.inverse_transform(preds_finetuned.predictions.argmax(axis=1))
accuracy_finetuned = accuracy_score(test_labels, predictions_finetuned)
precision_finetuned = precision_score(test_labels, predictions_finetuned, average='weighted')
recall_finetuned = recall_score(test_labels, predictions_finetuned, average='weighted')
f1_finetuned = f1_score(test_labels, predictions_finetuned, average='weighted')
finetuned_processing_time = time.time() - start_time

# Store results in a DataFrame
results = pd.DataFrame({
    "Model": ["Pre-trained BERT", "Fine-tuned BERT (skipping intermediate layers)"],
    "Accuracy": [accuracy_pretrained, accuracy_finetuned],
    "Precision": [precision_pretrained, precision_finetuned],
    "Recall": [recall_pretrained, recall_finetuned],
    "F1-score": [f1_pretrained, f1_finetuned],
    "Processing Time (seconds)": [pretrained_processing_time, finetuned_processing_time]
})

# Print results
print(results)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,1.624141
2,No log,1.591408
3,No log,1.592376


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batc

Epoch,Training Loss,Validation Loss
1,No log,1.605580
2,No log,1.641241
3,No log,1.613981


                                            Model  Accuracy  Precision  \
0                                Pre-trained BERT     0.325   0.251915   
1  Fine-tuned BERT (skipping intermediate layers)     0.250   0.195211   

   Recall  F1-score  Processing Time (seconds)  
0   0.325  0.250602                 588.745347  
1   0.250  0.198276                 510.935875  


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
import time
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder
import torch

# Load the dataset
data = pd.read_csv("customer.csv")
# Take a sample of 200 rows
data = data.sample(n=200, random_state=42)
# Preprocess the data
data = data.dropna(subset=['Ticket Description'])  # Drop rows with missing values in the target column
data = data[['Ticket Description', 'Ticket Type']]  # Select relevant columns

# Split the data into train and test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(data['Ticket Description'], data['Ticket Type'], test_size=0.2, random_state=42)

# Encode labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Tokenization
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True)

# Convert encodings to a format suitable for the trainer
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create datasets
train_dataset = MyDataset(train_encodings, train_labels_encoded)
eval_dataset = MyDataset(test_encodings, test_labels_encoded)

# Define BERT model (Pre-trained)
model_pretrained = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_encoder.classes_))

# Define training arguments for pre-trained model
training_args_pretrained = TrainingArguments(
    output_dir='./results_pretrained',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy='epoch',
    logging_dir='./logs_pretrained',
    num_train_epochs=3,
    learning_rate=2e-5,
    save_strategy='epoch',  # Set save strategy to 'epoch'
    load_best_model_at_end=True
)

# Define trainer for pre-trained model
trainer_pretrained = Trainer(
    model=model_pretrained,
    args=training_args_pretrained,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)

# Train and evaluate pre-trained model
start_time = time.time()
trainer_pretrained.train()
preds_pretrained = trainer_pretrained.predict(eval_dataset)
predictions_pretrained = label_encoder.inverse_transform(preds_pretrained.predictions.argmax(axis=1))
accuracy_pretrained = accuracy_score(test_labels, predictions_pretrained)
precision_pretrained = precision_score(test_labels, predictions_pretrained, average='weighted')
recall_pretrained = recall_score(test_labels, predictions_pretrained, average='weighted')
f1_pretrained = f1_score(test_labels, predictions_pretrained, average='weighted')
pretrained_processing_time = time.time() - start_time

# Define BERT model (Fine-tuned by skipping intermediate layers)
model_finetuned = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_encoder.classes_))

# Fine-tune by skipping intermediate layers
layers_to_include = [0, 2, 4]  # Layers to include (0-indexed)
encoder_layers = model_finetuned.bert.encoder.layer
layers_to_remove = [i for i in range(len(encoder_layers)) if i not in layers_to_include]
model_finetuned.bert.encoder.layer = torch.nn.ModuleList([layer for i, layer in enumerate(encoder_layers) if i in layers_to_include])

# Define training arguments for fine-tuned model
training_args_finetuned = TrainingArguments(
    output_dir='./results_finetuned',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy='epoch',
    logging_dir='./logs_finetuned',
    num_train_epochs=3,
    learning_rate=2e-5,
    save_strategy='epoch',  # Set save strategy to 'epoch'
    load_best_model_at_end=True
)

# Define trainer for fine-tuned model
trainer_finetuned = Trainer(
    model=model_finetuned,
    args=training_args_finetuned,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)

# Train and evaluate fine-tuned model
start_time = time.time()
trainer_finetuned.train()
preds_finetuned = trainer_finetuned.predict(eval_dataset)
predictions_finetuned = label_encoder.inverse_transform(preds_finetuned.predictions.argmax(axis=1))
accuracy_finetuned = accuracy_score(test_labels, predictions_finetuned)
precision_finetuned = precision_score(test_labels, predictions_finetuned, average='weighted')
recall_finetuned = recall_score(test_labels, predictions_finetuned, average='weighted')
f1_finetuned = f1_score(test_labels, predictions_finetuned, average='weighted')
finetuned_processing_time = time.time() - start_time

# Store results in a DataFrame
results = pd.DataFrame({
    "Model": ["Pre-trained BERT", "Fine-tuned BERT (skipping intermediate layers)"],
    "Accuracy": [accuracy_pretrained, accuracy_finetuned],
    "Precision": [precision_pretrained, precision_finetuned],
    "Recall": [recall_pretrained, recall_finetuned],
    "F1-score": [f1_pretrained, f1_finetuned],
    "Processing Time (seconds)": [pretrained_processing_time, finetuned_processing_time]
})

# Print results
print(results)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,1.628249
2,No log,1.630643
3,No log,1.629398


Checkpoint destination directory ./results_pretrained/checkpoint-20 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results_pretrained/checkpoint-40 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results_pretrained/checkpoint-60 already exists and is non-empty. Saving will proceed but saved results may be invalid.


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batc

Epoch,Training Loss,Validation Loss
1,No log,1.617752
2,No log,1.612645
3,No log,1.601656


Checkpoint destination directory ./results_finetuned/checkpoint-20 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results_finetuned/checkpoint-40 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results_finetuned/checkpoint-60 already exists and is non-empty. Saving will proceed but saved results may be invalid.


                                            Model  Accuracy  Precision  \
0                                Pre-trained BERT     0.225   0.106731   
1  Fine-tuned BERT (skipping intermediate layers)     0.300   0.090000   

   Recall  F1-score  Processing Time (seconds)  
0   0.225  0.144737                 616.980701  
1   0.300  0.138462                 175.606799  


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [12]:
## Now intermediate layer skipping
import pandas as pd
from sklearn.model_selection import train_test_split
import time
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder
import torch

# Load the dataset
data = pd.read_csv("customer.csv")

# Group the data by 'Ticket Type' and sample an equal number of rows from each category
grouped_data = data.groupby('Ticket Type', group_keys=False).apply(lambda x: x.sample(min(len(x), 40), random_state=42))

# Preprocess the data
grouped_data = grouped_data.dropna(subset=['Ticket Description'])  # Drop rows with missing values in the target column
grouped_data = grouped_data[['Ticket Description', 'Ticket Type']]  # Select relevant columns

# Split the data into train and test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(grouped_data['Ticket Description'], grouped_data['Ticket Type'], test_size=0.2, random_state=42)

# Encode labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Tokenization
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True)

# Convert encodings to a format suitable for the trainer
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create datasets
train_dataset = MyDataset(train_encodings, train_labels_encoded)
eval_dataset = MyDataset(test_encodings, test_labels_encoded)


# Define BERT model (Pre-trained)
model_pretrained = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_encoder.classes_))

# Define training arguments for pre-trained model
training_args_pretrained = TrainingArguments(
    output_dir='./results_pretrained',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy='epoch',
    logging_dir='./logs_pretrained',
    num_train_epochs=3,
    learning_rate=2e-5,
    save_strategy='epoch',  # Set save strategy to 'epoch'
    load_best_model_at_end=True
)

# Define trainer for pre-trained model
trainer_pretrained = Trainer(
    model=model_pretrained,
    args=training_args_pretrained,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)

# Train and evaluate pre-trained model
start_time = time.time()
trainer_pretrained.train()
preds_pretrained = trainer_pretrained.predict(eval_dataset)
predictions_pretrained = label_encoder.inverse_transform(preds_pretrained.predictions.argmax(axis=1))
accuracy_pretrained = accuracy_score(test_labels, predictions_pretrained)
precision_pretrained = precision_score(test_labels, predictions_pretrained, average='weighted')
recall_pretrained = recall_score(test_labels, predictions_pretrained, average='weighted')
f1_pretrained = f1_score(test_labels, predictions_pretrained, average='weighted')
pretrained_processing_time = time.time() - start_time


# Define BERT model (Fine-tuned by skipping intermediate layers)
model_finetuned = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_encoder.classes_))

# Fine-tune by skipping intermediate layers
layers_to_include = [0, 2, 4, 6, 8, 10]  # Include only even-numbered layers (0-indexed)
encoder_layers = model_finetuned.bert.encoder.layer
model_finetuned.bert.encoder.layer = torch.nn.ModuleList([layer for i, layer in enumerate(encoder_layers) if i in layers_to_include])

# Define training arguments for fine-tuned model
training_args_finetuned = TrainingArguments(
    output_dir='./results_finetuned',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy='epoch',
    logging_dir='./logs_finetuned',
    num_train_epochs=3,
    learning_rate=2e-5,
    save_strategy='epoch',  # Set save strategy to 'epoch'
    load_best_model_at_end=True
)

# Define trainer for fine-tuned model
trainer_finetuned = Trainer(
    model=model_finetuned,
    args=training_args_finetuned,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)

# Train and evaluate fine-tuned model
start_time = time.time()
trainer_finetuned.train()
preds_finetuned = trainer_finetuned.predict(eval_dataset)
predictions_finetuned = label_encoder.inverse_transform(preds_finetuned.predictions.argmax(axis=1))
accuracy_finetuned = accuracy_score(test_labels, predictions_finetuned)
precision_finetuned = precision_score(test_labels, predictions_finetuned, average='weighted')
recall_finetuned = recall_score(test_labels, predictions_finetuned, average='weighted')
f1_finetuned = f1_score(test_labels, predictions_finetuned, average='weighted')
finetuned_processing_time = time.time() - start_time

# Store results in a DataFrame
# Store results in a DataFrame
results = pd.DataFrame({
    "Model": ["Pre-trained BERT", "Fine-tuned BERT (skipping intermediate layers)"],
    "Accuracy": [accuracy_pretrained, accuracy_finetuned],
    "Precision": [precision_pretrained, precision_finetuned],
    "Recall": [recall_pretrained, recall_finetuned],
    "F1-score": [f1_pretrained, f1_finetuned],
    "Processing Time (seconds)": [pretrained_processing_time, finetuned_processing_time]
})
# Print results
print(results)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,1.675156
2,No log,1.671737
3,No log,1.666355


Checkpoint destination directory ./results_pretrained/checkpoint-20 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results_pretrained/checkpoint-40 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results_pretrained/checkpoint-60 already exists and is non-empty. Saving will proceed but saved results may be invalid.


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batc

Epoch,Training Loss,Validation Loss
1,No log,1.656739
2,No log,1.656206
3,No log,1.660780


Checkpoint destination directory ./results_finetuned/checkpoint-20 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results_finetuned/checkpoint-40 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results_finetuned/checkpoint-60 already exists and is non-empty. Saving will proceed but saved results may be invalid.


                                            Model  Accuracy  Precision  \
0                                Pre-trained BERT      0.10   0.036364   
1  Fine-tuned BERT (skipping intermediate layers)      0.15   0.058986   

   Recall  F1-score  Processing Time (seconds)  
0    0.10  0.051852                 602.671089  
1    0.15  0.073932                 329.443756  


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
## Run for whole data

## Now intermediate layer skipping
import pandas as pd
from sklearn.model_selection import train_test_split
import time
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder
import torch

# Load the dataset
data = pd.read_csv("customer.csv")

# Preprocess the data
data = data.dropna(subset=['Ticket Description'])  # Drop rows with missing values in the target column
data = data[['Ticket Description', 'Ticket Type']]  # Select relevant columns


# Group the data by 'Ticket Type' and sample an equal number of rows from each category
#grouped_data = data.groupby('Ticket Type', group_keys=False).apply(lambda x: x.sample(min(len(x), 40), random_state=42))

# Preprocess the data
#grouped_data = grouped_data.dropna(subset=['Ticket Description'])  # Drop rows with missing values in the target column
#grouped_data = grouped_data[['Ticket Description', 'Ticket Type']]  # Select relevant columns

# Split the data into train and test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(data['Ticket Description'], data['Ticket Type'], test_size=0.2, random_state=42)

# Encode labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Tokenization
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True)

# Convert encodings to a format suitable for the trainer
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create datasets
train_dataset = MyDataset(train_encodings, train_labels_encoded)
eval_dataset = MyDataset(test_encodings, test_labels_encoded)


# Define BERT model (Pre-trained)
model_pretrained = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_encoder.classes_))

# Define training arguments for pre-trained model
training_args_pretrained = TrainingArguments(
    output_dir='./results_pretrained',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy='epoch',
    logging_dir='./logs_pretrained',
    num_train_epochs=3,
    learning_rate=2e-5,
    save_strategy='epoch',  # Set save strategy to 'epoch'
    load_best_model_at_end=True
)

# Define trainer for pre-trained model
trainer_pretrained = Trainer(
    model=model_pretrained,
    args=training_args_pretrained,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)

# Train and evaluate pre-trained model
start_time = time.time()
trainer_pretrained.train()
preds_pretrained = trainer_pretrained.predict(eval_dataset)
predictions_pretrained = label_encoder.inverse_transform(preds_pretrained.predictions.argmax(axis=1))
accuracy_pretrained = accuracy_score(test_labels, predictions_pretrained)
precision_pretrained = precision_score(test_labels, predictions_pretrained, average='weighted')
recall_pretrained = recall_score(test_labels, predictions_pretrained, average='weighted')
f1_pretrained = f1_score(test_labels, predictions_pretrained, average='weighted')
pretrained_processing_time = time.time() - start_time


# Define BERT model (Fine-tuned by skipping intermediate layers)
model_finetuned = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_encoder.classes_))

# Fine-tune by skipping intermediate layers
layers_to_include = [0, 2, 4, 6, 8, 10]  # Include only even-numbered layers (0-indexed)
encoder_layers = model_finetuned.bert.encoder.layer
model_finetuned.bert.encoder.layer = torch.nn.ModuleList([layer for i, layer in enumerate(encoder_layers) if i in layers_to_include])

# Define training arguments for fine-tuned model
training_args_finetuned = TrainingArguments(
    output_dir='./results_finetuned',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy='epoch',
    logging_dir='./logs_finetuned',
    num_train_epochs=3,
    learning_rate=2e-5,
    save_strategy='epoch',  # Set save strategy to 'epoch'
    load_best_model_at_end=True
)

# Define trainer for fine-tuned model
trainer_finetuned = Trainer(
    model=model_finetuned,
    args=training_args_finetuned,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)

# Train and evaluate fine-tuned model
start_time = time.time()
trainer_finetuned.train()
preds_finetuned = trainer_finetuned.predict(eval_dataset)
predictions_finetuned = label_encoder.inverse_transform(preds_finetuned.predictions.argmax(axis=1))
accuracy_finetuned = accuracy_score(test_labels, predictions_finetuned)
precision_finetuned = precision_score(test_labels, predictions_finetuned, average='weighted')
recall_finetuned = recall_score(test_labels, predictions_finetuned, average='weighted')
f1_finetuned = f1_score(test_labels, predictions_finetuned, average='weighted')
finetuned_processing_time = time.time() - start_time

# Store results in a DataFrame
# Store results in a DataFrame
results = pd.DataFrame({
    "Model": ["Pre-trained BERT", "Fine-tuned BERT (skipping intermediate layers)"],
    "Accuracy": [accuracy_pretrained, accuracy_finetuned],
    "Precision": [precision_pretrained, precision_finetuned],
    "Recall": [recall_pretrained, recall_finetuned],
    "F1-score": [f1_pretrained, f1_finetuned],
    "Processing Time (seconds)": [pretrained_processing_time, finetuned_processing_time]
})
# Print results
print(results)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss
